In [11]:
import requests
import pandas as pd

In [12]:
API_KEY = ""

In [13]:
def get_yelp_reviews(business_id):
    """
    """
    url = f'https://api.yelp.com/v3/businesses/{business_id}/reviews?limit=50&sort_by=yelp_sort'
    headers = {'Authorization': f'Bearer {API_KEY}'}

    respuesta = requests.get(url, headers=headers)

    if respuesta.status_code == 200:
        # Leer archivo de respuesta JSON 
        reviews_data = respuesta.json()
        # Extraer `reviews`
        reviews = reviews_data['reviews']
        return reviews
    else:
        return None

In [35]:
df_darden_y_comp = pd.read_pickle('../data/df_darden_y_comp.pickle')

In [29]:
business_ids = df_darden_y_comp['business_id'].tolist()

In [30]:
b_id = business_ids[0]
b_id

'nSptExAJO8atirMyZbBh1g'

In [31]:
df_api_reviews = pd.DataFrame()

#for b_id in business_ids:
reviews = get_yelp_reviews(b_id)
# sleep(), rate limit

for i, r in enumerate(reviews):
    # Parse dia de la semana
    day = pd.to_datetime(r['time_created'])
    day_nombres = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    review = {
    'business_id': b_id,
    'review_id': r['id'],
    'user_id': r['user']['id'],
    'stars': float(r['rating']),
    'text': r['text'],
    'date': r['time_created'],
    'day': day_nombres[day.dayofweek]
    }

    df_api_reviews = pd.concat([df_api_reviews, pd.DataFrame(review, index=[i])])

df_api_reviews

,business_id,review_id,user_id,stars,text,date,day
0,nSptExAJO8atirMyZbBh1g,iUAZI9MSxsGV-4dMlph00g,qNcAN156WU_Ojr3yisCQxQ,1.0,Where do I begin. Don't eat here. First of all...,2024-04-20 12:30:09,Saturday
1,nSptExAJO8atirMyZbBh1g,jQIJNGIgBxkxhN2QSmQn4g,IiWcavXt0uCwQdiOaBh_XA,1.0,We were seated rather quickly. Our table was ...,2023-07-07 05:24:19,Friday
2,nSptExAJO8atirMyZbBh1g,dwEo7u4UkGi3-Kb7bBG55w,bi76Ih6tRv-ghYWFCJ0dag,3.0,Not bad. Came at 5:30 on a weeknight. The plac...,2023-01-05 17:13:35,Thursday


In [36]:
df = df_api_reviews.merge(df_darden_y_comp, on='business_id')

In [37]:
df

,business_id,review_id,user_id,stars,text,date,day,name,city,state,postal_code,coordinates,company
0,nSptExAJO8atirMyZbBh1g,iUAZI9MSxsGV-4dMlph00g,qNcAN156WU_Ojr3yisCQxQ,1.0,Where do I begin. Don't eat here. First of all...,2024-04-20 12:30:09,Saturday,Chili's,Mount Laurel,FL,08054,"39.923307,-74.9517928",brinker
1,nSptExAJO8atirMyZbBh1g,jQIJNGIgBxkxhN2QSmQn4g,IiWcavXt0uCwQdiOaBh_XA,1.0,We were seated rather quickly. Our table was ...,2023-07-07 05:24:19,Friday,Chili's,Mount Laurel,FL,08054,"39.923307,-74.9517928",brinker
2,nSptExAJO8atirMyZbBh1g,dwEo7u4UkGi3-Kb7bBG55w,bi76Ih6tRv-ghYWFCJ0dag,3.0,Not bad. Came at 5:30 on a weeknight. The plac...,2023-01-05 17:13:35,Thursday,Chili's,Mount Laurel,FL,08054,"39.923307,-74.9517928",brinker


In [54]:
df = df_darden_y_comp
company_unicos = df['company'].dropna().unique()

dfs = {}
for company in company_unicos:
    dfs[company] = df[df_darden_y_comp['company'] == company].drop('company', axis=1)

dfs.keys()

dict_keys(['brinker', 'darden', 'bloomin', 'texasroadhouse'])

In [56]:
tabla = 'reviews_'


for company, df in dfs.items():
    tabla = 'reviews_' + company
    print(tabla)

reviews_brinker
reviews_darden
reviews_bloomin
reviews_texasroadhouse
